In [266]:
# make usual imports
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime

In [2]:
# make list of Sundays since that's the day the NYTimes publishes the bestsellers list
start_date = datetime.date(2023, 3, 19)
end_date = datetime.date(2008, 6, 1)

sundays = []

while start_date > end_date:
    if start_date.weekday() == 6:  # Sunday has a weekday index of 6
        sundays.append([start_date.year, start_date.month, start_date.day])
    start_date -= datetime.timedelta(days=1)

In [133]:
# turn dates into strings
date_list = []
for sunday in sundays:
    year = sunday[0]
    month = sunday[1]
    day = sunday[2]
    date = f"{month}/{day}/{year}"
    date_list.append(date)

In [4]:
# make list of urls using list of Sundays
url_list = []
for element in sundays:
    year = str(element[0])
    if element[1]<=9:
        month = "0" + str(element[1])
    else:
        month = str(element[1])
    if element[2]<=9:
        day = "0" + str(element[2])
    else:
        day = str(element[2])
    url_list.append(f"https://www.nytimes.com/books/best-sellers/{year}/{month}/{day}/hardcover-fiction/")

In [5]:
# collect all html from urls in url_list
html_list = []   
for url in url_list:    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    results_url = soup.find_all("li", class_="css-13y32ub")
    html_list.append(results_url)

In [33]:
# turn all values into strings
html_list = [str(html) for html in html_list]

# make dictionary of dates and html 
date_and_html = dict(zip(date_list, html_list))

In [233]:
# make dictionary of dates, titles, authors, publishers, and descriptions
bestsellers = []
for date in date_list:
    book = date_and_html[date].split("<")
    name = [url.split(">")[1] for url in book if "itemprop=name" in url.replace('"', "")]
    author = [url.split(">")[1] for url in book if "itemprop=author" in url.replace('"', "")]
    publisher = [url.split(">")[1] for url in book if "itemprop=publisher" in url.replace('"', "")]
    description = [url.split(">")[1] for url in book if "itemprop=description" in url.replace('"', "")]
    dates_list2 = [date] * len(name)
    dict_date = {"date": dates_list2, "title": name, "author": author, 
                 "publisher": publisher, "description": description}
    bestsellers.append(dict_date)

In [234]:
# turn dictionary into dataframe
bestsellers_df = pd.DataFrame(bestsellers)

In [240]:
# check to see if the nested links are not the same length
wrong_list = []
for index, row in bestsellers_df.iterrows():
    if len(row["title"]) != len(row["author"]):
        wrong_list.append(index)
    elif len(row["title"]) != len(row["publisher"]):
        wrong_list.append(index)
    elif len(row["title"]) != len(row["description"]):
        wrong_list.append(index)
wrong_list

[]

In [244]:
# explode out nested lists and bring back together in new dataframe
bs_dates = bestsellers_df["date"]
bs_dates = bs_dates.explode("date")

bs_titles = bestsellers_df["title"]
bs_titles = bs_titles.explode("title")

bs_authors = bestsellers_df["author"]
bs_authors = bs_authors.explode("author")
bs_authors = bs_authors[bs_authors.str.startswith("by")]

bs_publishers = bestsellers_df["publisher"]
bs_publishers = bs_publishers.explode("publisher")

bs_description = bestsellers_df["description"]
bs_description = bs_description.explode("description")

df = pd.DataFrame({"date": bs_dates, "title": bs_titles, "author": bs_authors,
                  "publisher": bs_publishers, "description": bs_description})

# clean up author column
df["author"] = df["author"].str.replace("by", "")

In [246]:
# look at dataframe
df.head()

,date,title,author,publisher,description
0,3/19/2023,STORM WATCH,C.J. Box,Putnam,The 23rd book in the Joe Pickett series. Joe a...
1,3/19/2023,LESSONS IN CHEMISTRY,Bonnie Garmus,Doubleday,A scientist and single mother living in Califo...
2,3/19/2023,A DAY OF FALLEN NIGHT,Samantha Shannon,Bloomsbury,A prequel to “The Priory of the Orange Tree.” ...
3,3/19/2023,"TOMORROW, AND TOMORROW, AND TOMORROW",Gabrielle Zevin,Knopf,Two friends find their partnership challenged ...
4,3/19/2023,I HAVE SOME QUESTIONS FOR YOU,Rebecca Makkai,Viking,A film professor and podcaster is invited back...
...,...,...,...,...,...
10830,6/8/2008,INVINCIBLE,Troy Denning,Del Rey/Ballantine,"The New Jedi order battles Darth Caedus, Darth..."
10831,6/8/2008,BRIGHT SHINY MORNING,James Frey,Harper,"A novel, set in Los Angeles, by the author of ..."
10832,6/8/2008,THE ART OF RACING IN THE RAIN,Garth Stein,Harper,A Lab-terrier mix with great insight into the ...
10833,6/8/2008,TWENTY WISHES,Debbie Macomber,Mira,A widow who owns a bookstore on Blossom Street...


In [269]:
# double check no nan values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10835 entries, 0 to 10834
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         10835 non-null  object
 1   title        10835 non-null  object
 2   author       10835 non-null  object
 3   publisher    10835 non-null  object
 4   description  10835 non-null  object
dtypes: object(5)
memory usage: 423.4+ KB


In [ ]:
# find all rows missing authors
df.loc[df["author"].isnull()]

In [ ]:
# manually input missing values
df.iloc[[629],[2]] = "Beatriz Williams, Karen White, and Lauren Willig"
df.iloc[[6628],[2]] = "George R. R. Martin and Gardner Dozois"
df.iloc[[9279],[2]] = "Charlaine Harris and Toni L. P. Kelner"

In [268]:
df.to_csv(r'data/bestsellers_df.csv')